In [2]:
from utils.load_dataset import *
from utils.custom_utils import *

In [3]:
model_path_var = "./models/checkpoint_train_virtual_and_real.pth"
model_path_vtr = "./models/checkpoint_train_real_over_virtual.pth"
model_path_r = "./models/checkpoint_train_real.pth"
model_path_v = "./models/checkpoint_train_virtual.pth"

In [4]:
def load_model(path):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # create a Faster R-CNN model without pre-trained
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)

    num_classes = 3 # wheat or not(background)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # replace the pre-trained model's head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint['model'])
    # load the trained weights
    #model.load_state_dict(torch.load(path, map_location=device))
    model.eval()

    # move model to the right device
    model.to(device)
    return model, device

In [5]:
def take_prediction(prediction, threshold):
    boxes = prediction['boxes'].tolist()
    labels = prediction['labels'].tolist()
    scores = prediction['scores'].tolist()
    if len(boxes) == 0:
        return [([0,0,0,0],1,0.)]
    
    res = [t for t in zip(boxes,labels,scores) if t[2]>threshold]
    if len(res) == 0:
        res = [([0,0,0,0],1,0.)]
    return res

In [6]:
def visualize_prediction(dataset, list_imgs, model, device, path=None, thr=0.7):
    for l in list_imgs:
        img,target = dataset[l]
        with torch.no_grad():
            prediction = model([img.to(device)])
        p = take_prediction(prediction[0],thr)
        for bb,label,score in p:
            if label == 1:
                color = "green"
                text = f"no fallen: {score:.3f}"
            else:
                color = "red"
                text = f"fallen: {score:.3f}"
            x0,y0,x1,y1 = bb
            im = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
            draw = ImageDraw.Draw(im)
            draw.rectangle(((x0, y0),(x1,y1)), outline=color, width=3)
            draw.text((x0, y0), text)
        if path is None:
            ImageShow.show(im)
        else:
            im.save(f'snapshots/{path}{l}.png')

In [7]:
def classifier_performance(dataset, model, device, fallen_or_not, tr = 0.7):
    tn, tp, fn, fp = 0,0,0,0
    for im,target in tqdm(dataset):
        gt_labels = target['labels'].tolist()
        with torch.no_grad():
            prediction = model([im.to(device)])
        p = take_prediction(prediction[0], tr)
        pred_labels = [l for _,l,_ in p]
        len_gt_lab = len(gt_labels)
        len_pred_lab = len(pred_labels)
        fall_gt = [i for i in gt_labels if i==fallen_or_not]
        num_fall_gt = len(fall_gt)
        num_no_fall_gt = len_gt_lab - num_fall_gt

        fall_pred = [i for i in pred_labels if i==fallen_or_not] 
        num_fall_pred = len(fall_pred)
        num_no_fall_pred = len_pred_lab - num_fall_pred

        if num_fall_gt == num_fall_pred:
            tp += num_fall_gt
        elif num_fall_gt > num_fall_pred:
            tp += num_fall_pred
            fn += (num_fall_gt - num_fall_pred)
        else:
            tp += num_fall_gt
            fp += (num_fall_pred - num_fall_gt)
    return tp, fp, fn

# UP-FALL DETECTION

In [8]:
test_elderly = load_data("test_elderly.txt")
test_dataset = FallenPeople(test_elderly, "test_elderly", FallenPeople.valid_test_transform())

In [9]:
no_fall = 0
fall = 0
for t in test_dataset:
    if t[1]['labels'][0] == 1:
        no_fall += 1
    elif t[1]['labels'][0] == 2:
        fall += 1
print(no_fall)
print(fall)

141
272


In [10]:
def collate_fn(batch):
        return tuple(zip(*batch))
    
test_data_loader = DataLoader(
      test_dataset,
      batch_size=4,
      shuffle=False,
      collate_fn=collate_fn
    )

In [11]:
model_var, device = load_model(model_path_var)
model_vtr, _ = load_model(model_path_vtr)
model_r, _ = load_model(model_path_r)
model_v, _ = load_model(model_path_v)
test_list = [random.randint(0,len(test_dataset)-1) for i in range(5)]

# ONLY REAL

In [11]:
evaluate(model_r, test_data_loader, device=device)

creating index...
index created!
Test:  [  0/104]  eta: 0:00:40  model_time: 0.3274 (0.3274)  evaluator_time: 0.0043 (0.0043)  time: 0.3896  data: 0.0547  max mem: 2025
Test:  [100/104]  eta: 0:00:01  model_time: 0.3302 (0.3299)  evaluator_time: 0.0045 (0.0047)  time: 0.3930  data: 0.0550  max mem: 2025
Test:  [103/104]  eta: 0:00:00  model_time: 0.3295 (0.3276)  evaluator_time: 0.0045 (0.0046)  time: 0.3797  data: 0.0541  max mem: 2025
Test: Total time: 0:00:40 (0.3909 s / it)
Averaged stats: model_time: 0.3295 (0.3276)  evaluator_time: 0.0045 (0.0046)
Accumulating evaluation results...
DONE (t=0.12s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.165
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.397
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.087
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=med

### FALL DETECTION

In [12]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_r, device, 2,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 413/413 [00:44<00:00,  9.33it/s]


TP: 272	FN: 0	FP: 554
Recall: 1.00
Precision: 0.33
F1-score: 0.50


100%|██████████| 413/413 [00:44<00:00,  9.20it/s]


TP: 271	FN: 1	FP: 395
Recall: 1.00
Precision: 0.41
F1-score: 0.58


100%|██████████| 413/413 [00:45<00:00,  9.15it/s]


...

TP: 257	FN: 15	FP: 60
Recall: 0.94
Precision: 0.81
F1-score: 0.87


100%|██████████| 413/413 [00:45<00:00,  8.98it/s]


TP: 250	FN: 22	FP: 25
Recall: 0.92
Precision: 0.91
F1-score: 0.91


100%|██████████| 413/413 [00:45<00:00,  8.99it/s]

TP: 230	FN: 42	FP: 10
Recall: 0.85
Precision: 0.96
F1-score: 0.90


In [13]:
print(recall)
print(precision)
print(f1_score)

['1.00', '1.00', '0.99', '0.99', '0.97', '0.97', '0.96', '0.94', '0.92', '0.85']
['0.33', '0.41', '0.50', '0.56', '0.62', '0.67', '0.74', '0.81', '0.91', '0.96']
['0.50', '0.58', '0.67', '0.71', '0.76', '0.79', '0.83', '0.87', '0.91', '0.90']


### NO-FALL DETECTION

In [14]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_r, device, 1,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 413/413 [00:43<00:00,  9.48it/s]


TP: 141	FN: 0	FP: 1093
Recall: 1.00
Precision: 0.11
F1-score: 0.21


100%|██████████| 413/413 [00:43<00:00,  9.46it/s]


TP: 141	FN: 0	FP: 747
Recall: 1.00
Precision: 0.16
F1-score: 0.27


100%|██████████| 413/413 [00:45<00:00,  9.14it/s]


...

TP: 112	FN: 29	FP: 129
Recall: 0.79
Precision: 0.46
F1-score: 0.59


100%|██████████| 413/413 [00:45<00:00,  9.18it/s]


TP: 132	FN: 9	FP: 104
Recall: 0.94
Precision: 0.56
F1-score: 0.70


100%|██████████| 413/413 [00:44<00:00,  9.24it/s]

TP: 139	FN: 2	FP: 88
Recall: 0.99
Precision: 0.61
F1-score: 0.76


In [16]:
print(recall)
print(precision)
print(f1_score)

['1.00', '1.00', '1.00', '0.99', '0.89', '0.79', '0.73', '0.79', '0.94', '0.99']
['0.11', '0.16', '0.23', '0.28', '0.31', '0.35', '0.38', '0.46', '0.56', '0.61']
['0.21', '0.27', '0.37', '0.43', '0.46', '0.48', '0.50', '0.59', '0.70', '0.76']


## ONLY VIRTUAL

In [12]:
evaluate(model_v, test_data_loader, device=device)

creating index...
index created!
Test:  [  0/104]  eta: 0:00:44  model_time: 0.3402 (0.3402)  evaluator_time: 0.0091 (0.0091)  time: 0.4269  data: 0.0744  max mem: 2025
Test:  [100/104]  eta: 0:00:01  model_time: 0.3233 (0.3240)  evaluator_time: 0.0063 (0.0068)  time: 0.3896  data: 0.0542  max mem: 2025
Test:  [103/104]  eta: 0:00:00  model_time: 0.3233 (0.3218)  evaluator_time: 0.0057 (0.0067)  time: 0.3740  data: 0.0513  max mem: 2025
Test: Total time: 0:00:40 (0.3863 s / it)
Averaged stats: model_time: 0.3233 (0.3218)  evaluator_time: 0.0057 (0.0067)
Accumulating evaluation results...
DONE (t=0.19s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.070
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.179
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.034
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=med

### FALL DETECTION

In [13]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_v, device, 2,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 413/413 [00:43<00:00,  9.48it/s]


TP: 270	FN: 2	FP: 1612
Recall: 0.99
Precision: 0.14
F1-score: 0.25


100%|██████████| 413/413 [00:44<00:00,  9.27it/s]


TP: 267	FN: 5	FP: 861
Recall: 0.98
Precision: 0.24
F1-score: 0.38


100%|██████████| 413/413 [00:45<00:00,  9.16it/s]


...

TP: 43	FN: 229	FP: 8
Recall: 0.16
Precision: 0.84
F1-score: 0.27


100%|██████████| 413/413 [00:45<00:00,  9.17it/s]


TP: 9	FN: 263	FP: 1
Recall: 0.03
Precision: 0.90
F1-score: 0.06


100%|██████████| 413/413 [00:45<00:00,  9.12it/s]


ZeroDivisionError: division by zero

In [14]:
print(recall)
print(precision)
print(f1_score)

['0.99', '0.98', '0.85', '0.73', '0.55', '0.36', '0.16', '0.03']
['0.14', '0.24', '0.36', '0.51', '0.60', '0.73', '0.84', '0.90']
['0.25', '0.38', '0.51', '0.60', '0.57', '0.49', '0.27', '0.06']


### NO-FALL DETECTION

In [15]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_v, device, 1,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 413/413 [00:42<00:00,  9.69it/s]


TP: 141	FN: 0	FP: 3579
Recall: 1.00
Precision: 0.04
F1-score: 0.07


100%|██████████| 413/413 [00:43<00:00,  9.42it/s]


TP: 141	FN: 0	FP: 2303
Recall: 1.00
Precision: 0.06
F1-score: 0.11


100%|██████████| 413/413 [00:44<00:00,  9.31it/s]


...

TP: 140	FN: 1	FP: 398
Recall: 0.99
Precision: 0.26
F1-score: 0.41


100%|██████████| 413/413 [00:44<00:00,  9.31it/s]


TP: 141	FN: 0	FP: 332
Recall: 1.00
Precision: 0.30
F1-score: 0.46


100%|██████████| 413/413 [00:44<00:00,  9.31it/s]

TP: 141	FN: 0	FP: 293
Recall: 1.00
Precision: 0.32
F1-score: 0.49


In [16]:
print(recall)
print(precision)
print(f1_score)

['1.00', '1.00', '1.00', '0.99', '0.97', '0.97', '0.98', '0.99', '1.00', '1.00']
['0.04', '0.06', '0.09', '0.12', '0.15', '0.20', '0.24', '0.26', '0.30', '0.32']
['0.07', '0.11', '0.17', '0.22', '0.27', '0.33', '0.38', '0.41', '0.46', '0.49']


## VIRTUAL THAN REAL

In [17]:
evaluate(model_vtr, test_data_loader, device=device)

creating index...
index created!
Test:  [  0/104]  eta: 0:00:42  model_time: 0.3302 (0.3302)  evaluator_time: 0.0046 (0.0046)  time: 0.4043  data: 0.0438  max mem: 2025
Test:  [100/104]  eta: 0:00:01  model_time: 0.3232 (0.3228)  evaluator_time: 0.0048 (0.0050)  time: 0.3917  data: 0.0597  max mem: 2025
Test:  [103/104]  eta: 0:00:00  model_time: 0.3239 (0.3206)  evaluator_time: 0.0048 (0.0050)  time: 0.3791  data: 0.0586  max mem: 2025
Test: Total time: 0:00:39 (0.3839 s / it)
Averaged stats: model_time: 0.3239 (0.3206)  evaluator_time: 0.0048 (0.0050)
Accumulating evaluation results...
DONE (t=0.10s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.111
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.302
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.036
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=med

### FALL DETECTION

In [18]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_vtr, device, 2,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 413/413 [00:43<00:00,  9.49it/s]


TP: 272	FN: 0	FP: 1038
Recall: 1.00
Precision: 0.21
F1-score: 0.34


100%|██████████| 413/413 [00:43<00:00,  9.41it/s]


TP: 272	FN: 0	FP: 730
Recall: 1.00
Precision: 0.27
F1-score: 0.43


100%|██████████| 413/413 [00:44<00:00,  9.31it/s]


...

TP: 248	FN: 24	FP: 121
Recall: 0.91
Precision: 0.67
F1-score: 0.77


100%|██████████| 413/413 [00:44<00:00,  9.24it/s]


TP: 228	FN: 44	FP: 61
Recall: 0.84
Precision: 0.79
F1-score: 0.81


100%|██████████| 413/413 [00:44<00:00,  9.21it/s]

TP: 179	FN: 93	FP: 23
Recall: 0.66
Precision: 0.89
F1-score: 0.76


In [19]:
print(recall)
print(precision)
print(f1_score)

['1.00', '1.00', '1.00', '1.00', '1.00', '0.98', '0.94', '0.91', '0.84', '0.66']
['0.21', '0.27', '0.37', '0.43', '0.49', '0.54', '0.61', '0.67', '0.79', '0.89']
['0.34', '0.43', '0.53', '0.60', '0.65', '0.70', '0.74', '0.77', '0.81', '0.76']


### NO-FALL DETECTION

In [20]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_vtr, device, 1,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 413/413 [00:43<00:00,  9.54it/s]


TP: 141	FN: 0	FP: 1316
Recall: 1.00
Precision: 0.10
F1-score: 0.18


100%|██████████| 413/413 [00:43<00:00,  9.41it/s]


TP: 140	FN: 1	FP: 856
Recall: 0.99
Precision: 0.14
F1-score: 0.25


100%|██████████| 413/413 [00:44<00:00,  9.26it/s]


...

TP: 91	FN: 50	FP: 67
Recall: 0.65
Precision: 0.58
F1-score: 0.61


100%|██████████| 413/413 [00:44<00:00,  9.20it/s]


TP: 105	FN: 36	FP: 64
Recall: 0.74
Precision: 0.62
F1-score: 0.68


100%|██████████| 413/413 [00:44<00:00,  9.19it/s]

TP: 127	FN: 14	FP: 97
Recall: 0.90
Precision: 0.57
F1-score: 0.70


In [21]:
print(recall)
print(precision)
print(f1_score)

['1.00', '0.99', '0.97', '0.89', '0.79', '0.69', '0.65', '0.65', '0.74', '0.90']
['0.10', '0.14', '0.22', '0.29', '0.35', '0.42', '0.50', '0.58', '0.62', '0.57']
['0.18', '0.25', '0.36', '0.44', '0.49', '0.52', '0.57', '0.61', '0.68', '0.70']


## VIRTUAL AND REAL

In [22]:
evaluate(model_var, test_data_loader, device=device)

creating index...
index created!
Test:  [  0/104]  eta: 0:00:43  model_time: 0.3250 (0.3250)  evaluator_time: 0.0039 (0.0039)  time: 0.4212  data: 0.0631  max mem: 2025
Test:  [100/104]  eta: 0:00:01  model_time: 0.3222 (0.3208)  evaluator_time: 0.0045 (0.0047)  time: 0.3886  data: 0.0554  max mem: 2025
Test:  [103/104]  eta: 0:00:00  model_time: 0.3244 (0.3187)  evaluator_time: 0.0044 (0.0047)  time: 0.3756  data: 0.0540  max mem: 2025
Test: Total time: 0:00:39 (0.3824 s / it)
Averaged stats: model_time: 0.3244 (0.3187)  evaluator_time: 0.0044 (0.0047)
Accumulating evaluation results...
DONE (t=0.09s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.328
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.078
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=med

### FALL DETECTION

In [23]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_var, device, 2,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 413/413 [00:44<00:00,  9.36it/s]


TP: 272	FN: 0	FP: 1017
Recall: 1.00
Precision: 0.21
F1-score: 0.35


100%|██████████| 413/413 [00:44<00:00,  9.29it/s]


TP: 272	FN: 0	FP: 682
Recall: 1.00
Precision: 0.29
F1-score: 0.44


100%|██████████| 413/413 [00:44<00:00,  9.19it/s]


...

TP: 267	FN: 5	FP: 162
Recall: 0.98
Precision: 0.62
F1-score: 0.76


100%|██████████| 413/413 [00:44<00:00,  9.29it/s]


TP: 262	FN: 10	FP: 121
Recall: 0.96
Precision: 0.68
F1-score: 0.80


100%|██████████| 413/413 [00:44<00:00,  9.28it/s]

TP: 233	FN: 39	FP: 79
Recall: 0.86
Precision: 0.75
F1-score: 0.80


In [24]:
print(recall)
print(precision)
print(f1_score)

['1.00', '1.00', '1.00', '0.99', '0.99', '0.99', '0.98', '0.98', '0.96', '0.86']
['0.21', '0.29', '0.38', '0.43', '0.48', '0.54', '0.58', '0.62', '0.68', '0.75']
['0.35', '0.44', '0.55', '0.60', '0.65', '0.70', '0.73', '0.76', '0.80', '0.80']


### NO-FALL DETECTION

In [25]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_var, device, 1,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 413/413 [00:43<00:00,  9.42it/s]


TP: 141	FN: 0	FP: 724
Recall: 1.00
Precision: 0.16
F1-score: 0.28


100%|██████████| 413/413 [00:44<00:00,  9.27it/s]


TP: 111	FN: 30	FP: 325
Recall: 0.79
Precision: 0.25
F1-score: 0.38


100%|██████████| 413/413 [00:44<00:00,  9.21it/s]


...

TP: 25	FN: 116	FP: 38
Recall: 0.18
Precision: 0.40
F1-score: 0.25


100%|██████████| 413/413 [00:44<00:00,  9.33it/s]


TP: 33	FN: 108	FP: 30
Recall: 0.23
Precision: 0.52
F1-score: 0.32


100%|██████████| 413/413 [00:44<00:00,  9.33it/s]

TP: 64	FN: 77	FP: 49
Recall: 0.45
Precision: 0.57
F1-score: 0.50


In [26]:
print(recall)
print(precision)
print(f1_score)

['1.00', '0.79', '0.64', '0.54', '0.44', '0.33', '0.25', '0.18', '0.23', '0.45']
['0.16', '0.25', '0.38', '0.45', '0.45', '0.44', '0.44', '0.40', '0.52', '0.57']
['0.28', '0.38', '0.47', '0.49', '0.44', '0.38', '0.32', '0.25', '0.32', '0.50']
